<a href="https://colab.research.google.com/github/manasvi2909/Spotify-Music-Recommender/blob/main/SpotifyMusicRecommender_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommender Demo Notebook

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [5]:
from google.colab import files
uploaded = files.upload()

Saving SpotifyAudioFeaturesApril2019.csv to SpotifyAudioFeaturesApril2019.csv


In [6]:
# Load dataset
df = pd.read_csv("SpotifyAudioFeaturesApril2019.csv")
df.head()

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.118,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.371,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.382,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.641,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.928,0


In [7]:
# Preprocess
features = ["danceability","energy","loudness","speechiness",
            "acousticness","instrumentalness","liveness","valence","tempo"]
X = df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Pick a seed song
seed_query = "Shape of You"  # change this to anything

matches = df[
    df["track_name"].str.contains(seed_query, case=False, na=False) |
    df["artist_name"].str.contains(seed_query, case=False, na=False)
]

matches = matches.head(10)

display(matches[["track_name", "artist_name", "track_id"]])

assert not matches.empty, "No matches for your query. Try a different song/artist."

seed_idx = matches.index[0]
print("Using seed:", df.loc[seed_idx, "track_name"], "—", df.loc[seed_idx, "artist_name"])

,track_name,artist_name,track_id
48080,Shape of You (feat. Crysa),Conkarah,0KNm3Rpi7AYHz7FlrCHLXa
48991,Shape of You,Night Water Project,7K90a0scH4d5slh9nkXBqU
49711,Shape of Your Face,Bone Apple Tea,7pouBXPRnelSpXExnfAkGh
56731,Shape Of You,Amaia Romero,0vWlNlKFDeXSpSc5sbYpW1
65603,Shape of You,Trevor Nelson,7aSqU9YKmMkAck42CpbPen
71873,Shape of You,Gonzalo Porta,3w1fTKQtawG8Wo7LKgYDmz
94485,Shape of You - Recorded at Spotify Studios NYC,Yo-Yo Ma,5RMmI12HWgdBYtJHE1wEgY
126260,Despacito x Shape Of You,Pentatonix,5cj54CVe4pQZ9cUKgbsZrG


Using seed: Shape of You (feat. Crysa) — Conkarah


In [10]:
# Use the first match
seed_idx = matches.index[0]

In [11]:
# Build recommender
model = NearestNeighbors(n_neighbors=11, metric="cosine")
model.fit(X_scaled)

distances, indices = model.kneighbors([X_scaled[seed_idx]])

In [12]:
# Show recommendations
recs = df.iloc[indices[0][1:]].copy()
recs["distance"] = distances[0][1:]
recs["similarity"] = 1 - recs["distance"]
recs[["track_name","artist_name","similarity"]]

,track_name,artist_name,similarity
25497,Waist Deep,Cryptic Wisdom,0.993815
91697,every night,F430,0.989659
8388,Last Dance,Outlaw,0.988439
68543,On Errythang,Rnb Base,0.985793
124384,Summertime Magic,Childish Gambino,0.984058
27813,Know U Anymore (feat. Sarah Hyland) - ye. Remix,BoTalks,0.984038
21591,Phat,Andre Nickatina,0.984007
52979,Day Job (Instrumental),Sir Froderick,0.983373
5494,Chizzle (feat. Sly Pyper & Daz Dillinger),Snoop Dogg,0.981160
113385,Can't Hang,Preme,0.980470
